# Classify 100 numbers into 5 Categories with PyTorch

by Ding

This notebook shows how to use a neural network for a (simple) classification problem.

We want machine to learn to classify [0, 0.2) into Class 0, [0.2, 0.4) into Class 1, [0.4, 0.6) into Class 2, [0.6, 0.8) into Class 3 and [0.8, 1) into Class 4.

In [5]:
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import torch 
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.utils.data as Data
from torch.autograd import Variable

In [6]:
nums = np.arange(1, 101)
nums

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100])

In [7]:
nums_1 = nums.reshape(100, 1)

In [8]:
def normalize(x):
    x1 = x - min(x)
    x2 = max(x) - min(x)
    return x1/x2

nums_norm = normalize(nums_1) # Really need to normalize it?

In [9]:
labels = np.zeros([100,1])

labels[20:40] = 1
labels[40:60] = 2
labels[60:80] = 3
labels[80:100] = 4

In [10]:
x_train, x_test, y_train, y_test = train_test_split(nums_norm, labels, test_size=0.2)

In [11]:
x_train = torch.from_numpy(x_train).type(torch.FloatTensor)
x_test = torch.from_numpy(x_test).type(torch.FloatTensor)
y_train = torch.from_numpy(y_train).type(torch.LongTensor)
y_test = torch.from_numpy(y_test).type(torch.LongTensor)

In [12]:
y_train.shape, y_test.shape

(torch.Size([80, 1]), torch.Size([20, 1]))

In [13]:
y_train = torch.squeeze(y_train) # It has to fit the dimension.
y_test = torch.squeeze(y_test)

In [14]:
y_train.shape, y_test.shape

(torch.Size([80]), torch.Size([20]))

In [15]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(1,8)
        self.fc2 = nn.Linear(8,5)
        
    def forward(self,x):
        x = self.fc1(x)
        x = F.tanh(x)
        x = self.fc2(x)
        return x
              
    def predict(self,x):
        pred = F.softmax(self.forward(x))
        pred = pred.tolist()
        ans = []
        for t in pred:
          ans.append(t.index(max(t))) # Find the position (index) of the largest probability among the five predicted probabilities
        return torch.tensor(ans)

In [16]:
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [17]:
epochs = 250
losses = []
for i in range(epochs):
    y_pred = model.forward(x_train)
    loss = criterion(y_pred, y_train)
    losses.append(loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
print("Accuracy:")
print(accuracy_score(model.predict(x_train), y_train))    

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1794: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Accuracy:
0.9625


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


In [18]:
print("Observation:")
print(y_test)
print("Prediction:")
print(model.predict(x_test))

print("Accuracy:")
print(accuracy_score(model.predict(x_test), y_test))

Observation:
tensor([2, 4, 3, 3, 4, 3, 1, 0, 0, 3, 1, 1, 2, 2, 4, 3, 3, 2, 4, 0])
Prediction:
tensor([2, 4, 3, 3, 4, 3, 1, 0, 0, 3, 1, 1, 2, 2, 4, 4, 3, 2, 4, 0])
Accuracy:
0.95


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1794: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  
